In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [4]:
from sklearn.neural_network import MLPRegressor

In [5]:
total_data = pd.read_csv('Combined_Roto_NBA_18-19.csv')

/Users/jordanlevy/Desktop/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
total_data.columns

Index(['Date', 'Name', 'Salaries', 'Team', 'Position', 'Opponent',
       'Unnamed: 6', 'Projected Minutes', 'DvP', 'DvPRank', 'O/U', 'Line',
       'Total', 'Movement', 'Ceiling', 'Floor', 'Projection', 'Pts/$/K', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'FD_PTS'],
      dtype='object')

In [7]:
feat_data = total_data[['Date', 'Position', 'Salaries', 'Line', 'Projected Minutes', 'Ceiling', 'Floor', 'Projection', 'FD_PTS']]

In [8]:
feat_data = feat_data[feat_data['Line'] != '\xa0'].reset_index(drop=True)#['Line'].str.strip(' ')

In [62]:
feat_data['Line'] = feat_data['Line'].astype('int')

In [63]:
mean_data = feat_data.groupby('Date').transform(np.mean)
std_data = feat_data.groupby('Date').transform(np.std)

In [64]:
feat_data.head()

,Date,Position,Salaries,Line,Projected Minutes,Ceiling,Floor,Projection,FD_PTS
0,2018-10-16,PF,6400,180,33.0,37.61,18.53,28.07,11.7
1,2018-10-16,C,10400,180,33.0,59.02,33.20,46.11,42.0
2,2018-10-16,SG,5900,-215,32.0,34.31,18.07,26.19,19.0
3,2018-10-16,SF,6500,180,33.0,39.30,18.92,29.11,22.2
4,2018-10-16,PG,8400,-215,33.0,46.72,28.64,37.68,19.3


In [65]:
feat_scale_data = (feat_data.iloc[:,2:] - mean_data) / std_data 

In [66]:
feat_scale_data['Position'] = feat_data['Position'].values

In [67]:
feat_scale_data.head()

,Salaries,Line,Projected Minutes,Ceiling,Floor,Projection,FD_PTS,Position
0,0.359145,0.364787,1.057416,0.403134,0.369699,0.398658,-0.690765,PF
1,2.166457,0.364787,1.057416,1.968046,1.784520,1.937222,1.249517,C
2,0.133231,-0.008605,0.931302,0.161928,0.325335,0.238320,-0.223304,SG
3,0.404328,0.364787,1.057416,0.526660,0.407312,0.487356,-0.018390,SF
4,1.262801,-0.008605,1.057416,1.069007,1.344739,1.218259,-0.204093,PG


In [152]:
mlp = MLPRegressor(hidden_layer_sizes = (100,), early_stopping=True)

In [158]:
X = feat_scale_data[['Salaries', 'Projection', 'Line', 'Projected Minutes', 'Ceiling', 'Floor']]

In [159]:
y = feat_scale_data[['FD_PTS']]

In [160]:
import warnings
warnings.filterwarnings("ignore")

In [161]:
score_arr = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state =np.random)
    mlp.fit(X_train, y_train)
    score = mlp.score(X_test, y_test)
    score_arr.append(score)
    print(i+1)
np.mean(score_arr)

1
2
3
4
5


0.5339158307420084

In [157]:
mlp.fit(X_train, y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [123]:
mlp.predict(X_test)

array([-0.04480239, -0.09836436, -0.55119022, ..., -0.14659501,
        0.18650428,  1.03417749])

In [124]:
mlp.score(X_test, y_test)

0.5352854664902953

In [88]:
ohe = OneHotEncoder(sparse=False)

In [60]:
X = ohe.fit_transform(total_data[['Position']])

In [61]:
X

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])